## **Team Members**


**ATHARAVA ANIL CHANDA - PES2UG20CS075**

**VIJAY J              - PES2UG20CS815**

**CHINMAY GOWDA        - PES2UG20CS902**



##**Importing Libraries**

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## **Reading the Dataset**

In [ ]:
data = pd.read_csv('/content/IMDB top 1000.csv')

In [ ]:
data.head(5)

,Unnamed: 0,Title,Certificate,Duration,Genre,Rate,Metascore,Description,Cast,Info
0,0,1. The Shawshank Redemption (1994),R,142 min,Drama,9.3,80.0,Two imprisoned men bond over a number of years...,"Director: Frank Darabont | Stars: Tim Robbins,...","Votes: 2,295,987 | Gross: $28.34M"
1,1,2. The Godfather (1972),R,175 min,"Crime, Drama",9.2,100.0,The aging patriarch of an organized crime dyna...,Director: Francis Ford Coppola | Stars: Marlon...,"Votes: 1,584,782 | Gross: $134.97M"
2,2,3. The Dark Knight (2008),PG-13,152 min,"Action, Crime, Drama",9.0,84.0,When the menace known as the Joker wreaks havo...,Director: Christopher Nolan | Stars: Christian...,"Votes: 2,260,649 | Gross: $534.86M"
3,3,4. The Godfather: Part II (1974),R,202 min,"Crime, Drama",9.0,90.0,The early life and career of Vito Corleone in ...,Director: Francis Ford Coppola | Stars: Al Pac...,"Votes: 1,107,253 | Gross: $57.30M"
4,4,5. The Lord of the Rings: The Return of the Ki...,PG-13,201 min,"Action, Adventure, Drama",8.9,94.0,Gandalf and Aragorn lead the World of Men agai...,"Director: Peter Jackson | Stars: Elijah Wood, ...","Votes: 1,614,369 | Gross: $377.85M"


##**PreProcessing the Data**

### **Text Cleaning**

In [ ]:
text = data['Description'].head(500)
text1 = data['Title'].head(500)
originalText = text
originalText = text1
print(text)
print(text1)

0      Two imprisoned men bond over a number of years...
1      The aging patriarch of an organized crime dyna...
2      When the menace known as the Joker wreaks havo...
3      The early life and career of Vito Corleone in ...
4      Gandalf and Aragorn lead the World of Men agai...
                             ...                        
495    A prep school student needing money agrees to ...
496    A kindhearted street urchin and a power-hungry...
497    New Orleans District Attorney Jim Garrison dis...
498    A waifish prostitute wanders the streets of Ro...
499    A war-hardened general, egged on by his ambiti...
Name: Description, Length: 500, dtype: object
0                     1. The Shawshank Redemption (1994)
1                                2. The Godfather (1972)
2                              3. The Dark Knight (2008)
3                       4. The Godfather: Part II (1974)
4      5. The Lord of the Rings: The Return of the Ki...
                             ...          

In [ ]:
def clean_text(text):
    # Convert to string
    text = str(text)
    
    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)
    
    return text

# Clean the text in the Data using the clean_text() function
cleaned_text = data['Description'].head(500).apply(clean_text)
cleaned_text1 = data['Title'].head(500).apply(clean_text)
# Print the cleaned text
print(cleaned_text)
print(cleaned_text1)


0      two imprisoned men bond number years finding s...
1      aging patriarch organized crime dynasty transf...
2      menace known joker wreaks havoc chaos people g...
3      early life career vito corleone new york city ...
4      gandalf aragorn lead world men saurons army dr...
                             ...                        
495    prep school student needing money agrees babys...
496    kindhearted street urchin powerhungry grand vi...
497    new orleans district attorney jim garrison dis...
498    waifish prostitute wanders streets rome lookin...
499    warhardened general egged ambitious wife works...
Name: Description, Length: 500, dtype: object
0        shawshank redemption
1                   godfather
2                 dark knight
3           godfather part ii
4      lord rings return king
                ...          
495               scent woman
496                   aladdin
497                       jfk
498            nights cabiria
499              throne blo

### **Stemming and Lemmatization**

In [ ]:
# Stemming
ps = PorterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    #token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(ps.stem(word)) #STEMMING
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatizeSentence(sentence):
    token_words=word_tokenize(sentence)
    #token_words
    lem_sentence=[]
    for word in token_words:
        word1 = lemmatizer.lemmatize(word, pos = "n")
        word2 = lemmatizer.lemmatize(word1, pos = "v")
        word3 = lemmatizer.lemmatize(word2, pos = ("a"))
        lem_sentence.append(word1)
        lem_sentence.append(" ")
    return "".join(lem_sentence)

In [ ]:
new_filtered_Sentence=[]
for i in cleaned_text:
    j = stemSentence(i)
    new_filtered_Sentence.append(j) 

final_filtered_Sentence=[]
for i in new_filtered_Sentence:
    j = lemmatizeSentence(i) #LEMMATISATION
    final_filtered_Sentence.append(j) 

final_filtered_Sentence

['two imprison men bond number year find solac eventu redempt act common decenc ',
 'age patriarch organ crime dynasti transfer control clandestin empir reluct son ',
 'menac known joker wreak havoc chao peopl gotham batman must accept one greatest psycholog physic test abil fight injustic ',
 'earli life career vito corleon new york citi portray son michael expand tighten grip famili crime syndic ',
 'gandalf aragorn lead world men sauron armi draw gaze frodo sam approach mount doom one ring ',
 'live two mob hitman boxer gangster wife pair diner bandit intertwin four tale violenc redempt ',
 'germanoccupi poland world war ii industrialist oskar schindler gradual becom concern jewish workforc wit persecut nazi ',
 'juri holdout attempt prevent miscarriag justic forc colleagu reconsid evid ',
 'thief steal corpor secret use dreamshar technolog given invers task plant idea mind ceo ',
 'insomniac offic worker devilmaycar soapmak form underground fight club evolv someth much much ',
 'me

### **Vectorization**

In [ ]:
# Vectorize the preprocessed data
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(final_filtered_Sentence)

# Print the shape of the vectors
print(vectors.shape)

(500, 2508)


### **Free Text Query using Inverted Index**

In [ ]:
# Define a function to create an inverted index with rank
def create_inverted_index(data):
    inverted_index = {}
    for i, row in data.iterrows():
        # Combine relevant fields into a single string
        text = f"{row['Title']} {row['Genre']} {row['Cast']} {row['Metascore']}"

        # Split text into individual terms
        terms = text.lower().split()

        # Compute the rank of the document (in this case, IMDb rating)
        rank = row['Rate']

        # Add terms and rank to the inverted index
        for term in terms:
            if term not in inverted_index:
                inverted_index[term] = []
            inverted_index[term].append((i, rank))
    return inverted_index

# Create an inverted index with rank for the IMDb top 1000 movies dataset
inverted_index = create_inverted_index(data)

# Define a function to perform a free text query with rank using the inverted index
def free_text_query(query, inverted_index):
    terms = query.lower().split()
    results = {}
    for term in terms:
        if term in inverted_index:
            for doc_id, rank in inverted_index[term]:
                if doc_id not in results:
                    results[doc_id] = 0
                results[doc_id] += rank
    return sorted(results.items(), key=lambda x: x[1], reverse=True)

# Perform a free text query for "action thriller directed by Christopher Nolan"
results = free_text_query("action thriller directed by Christopher Nolan", inverted_index)
for result in results:
    title = data.loc[result[0], 'Title']
    rating = data.loc[result[0], 'Rate']
    print(f"Title: {title} | Rating: {rating} | Rank: {result[1]}")

Title: 69. Memento (2000) | Rating: 8.4 | Rank: 25.200000000000003
Title: 3. The Dark Knight (2008) | Rating: 9.0 | Rank: 18.0
Title: 9. Inception (2010) | Rating: 8.8 | Rank: 17.6
Title: 21. Interstellar (2014) | Rating: 8.6 | Rank: 17.2
Title: 36. The Prestige (2006) | Rating: 8.5 | Rank: 17.0
Title: 63. The Dark Knight Rises (2012) | Rating: 8.4 | Rank: 16.8
Title: 119. North by Northwest (1959) | Rating: 8.3 | Rank: 16.6
Title: 154. Batman Begins (2005) | Rating: 8.2 | Rank: 16.4
Title: 20. Parasite (2019) | Rating: 8.6 | Rank: 8.6
Title: 28. The Silence of the Lambs (1991) | Rating: 8.6 | Rank: 8.6
Title: 33. Joker (2019) | Rating: 8.5 | Rank: 8.5
Title: 37. The Departed (2006) | Rating: 8.5 | Rank: 8.5
Title: 41. The Usual Suspects (1995) | Rating: 8.5 | Rank: 8.5
Title: 50. Psycho (1960) | Rating: 8.5 | Rank: 8.5
Title: 47. Back to the Future (1985) | Rating: 8.5 | Rank: 8.5
Title: 67. The Lives of Others (2006) | Rating: 8.4 | Rank: 8.4
Title: 80. Rear Window (1954) | Rating: 8

## **Inverted Index**

In [ ]:
# Create an empty dictionary for the inverted index
inverted_index = {}

# Loop over all documents in the dataset
for doc_id, doc_text in enumerate(final_filtered_Sentence):
    
    # Tokenize the document text
    doc_tokens = doc_text.split()
    
    # Loop over all tokens in the document
    for token in doc_tokens:
        
        # Check if the token is already in the inverted index
        if token in inverted_index:
            
            # Add the document ID to the postings list for the token
            inverted_index[token].append(doc_id)
            
        else:
            
            # Create a new postings list for the token
            inverted_index[token] = [doc_id]
inverted_index

{'two': [0,
  5,
  12,
  21,
  26,
  28,
  35,
  55,
  56,
  63,
  82,
  112,
  115,
  159,
  163,
  164,
  167,
  171,
  173,
  177,
  179,
  187,
  200,
  224,
  228,
  232,
  235,
  248,
  256,
  265,
  269,
  279,
  280,
  283,
  293,
  296,
  307,
  311,
  340,
  365,
  387,
  407,
  411,
  440,
  465,
  487],
 'imprison': [0, 67, 113, 258, 289, 311, 411],
 'men': [0, 4, 12, 82, 83, 158, 171, 182, 252, 302, 402],
 'bond': [0, 223, 248, 248, 363, 363, 463, 463],
 'number': [0, 175],
 'year': [0,
  46,
  62,
  67,
  70,
  90,
  106,
  140,
  161,
  193,
  204,
  204,
  233,
  264,
  283,
  291,
  326,
  336,
  366,
  426,
  436,
  466],
 'find': [0,
  12,
  55,
  60,
  66,
  67,
  67,
  69,
  69,
  71,
  87,
  99,
  100,
  105,
  114,
  121,
  122,
  142,
  166,
  171,
  189,
  196,
  206,
  226,
  262,
  270,
  278,
  279,
  284,
  287,
  298,
  305,
  314,
  318,
  319,
  326,
  327,
  346,
  346,
  347,
  370,
  375,
  393,
  398,
  405,
  414,
  418,
  419,
  426,
  427,
  446,


## **Retrieval based on Similarity Index**

In [ ]:
# Define the vectorizer
vectorizer = TfidfVectorizer()

# Vectorize the text data
vectors = vectorizer.fit_transform(final_filtered_Sentence)

# Calculate the cosine similarity matrix
cosine_similarities = cosine_similarity(vectors)

# Get input query from user
query = input("Enter your query: ")

# Vectorize the query
query_vector = vectorizer.transform([query])

# Calculate the cosine similarities between the query vector and all document vectors
query_cosine_similarities = cosine_similarity(query_vector, vectors)

# Get the indices of the documents with the highest cosine similarities to the query
similar_doc_indices = query_cosine_similarities.argsort()[0][::-1][:10]

# Print the titles and scores of the top 10 similar documents
for i, index in enumerate(similar_doc_indices):
    print(f"Rank: {i+1}")
    print(f"Title: {data['Title'][index]}")
    print(f"Score: {query_cosine_similarities[0][index]}")
    print("\n")

Enter your query: dark knight
Rank: 1
Title: 175. Monty Python and the Holy Grail (1975)
Score: 0.18946147146101014


Rank: 2
Title: 136. Drishyam (2015)
Score: 0.1802737961170131


Rank: 3
Title: 362. Sin City (2005)
Score: 0.16980822829732872


Rank: 4
Title: 362. Sin City (2005)
Score: 0.16980822829732872


Rank: 5
Title: 29. Star Wars: Episode IV - A New Hope (1977)
Score: 0.1663910857112311


Rank: 6
Title: 153. Black (2005)
Score: 0.16220792902677708


Rank: 7
Title: 11. The Lord of the Rings: The Fellowship of the Ring (2001)
Score: 0.15075333332254146


Rank: 8
Title: 224. Harry Potter and the Deathly Hallows: Part 2 (2011)
Score: 0.14861064986050757


Rank: 9
Title: 110. Star Wars: Episode VI - Return of the Jedi (1983)
Score: 0.11325732544969244


Rank: 10
Title: 302. Throne of Blood (1957)
Score: 0.0




### **Boolean Query Using Inverted index**

In [ ]:
import pandas as pd
import re


# Clean and preprocess the text
def cleaned_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Create the inverted index
inverted_index = {}
for index, row in data.iterrows():
    title = cleaned_text(row['Title'])
    words = title.split()
    doc_id = index
    for word in words:
        if word in inverted_index:
            inverted_index[word].append(doc_id)
        else:
            inverted_index[word] = [doc_id]

# Define the functions for boolean retrieval
def boolean_and(query):
    doc_ids = set(inverted_index[query[0]])
    for word in query:
        doc_ids = doc_ids.intersection(set(inverted_index[word]))
    return doc_ids

def boolean_or(query):
    doc_ids = set()
    for word in query:
        if word in inverted_index:
            doc_ids = doc_ids.union(set(inverted_index[word]))
    return doc_ids

def boolean_not(query):
    doc_ids = set(range(len(data)))
    not_query = boolean_and(query[1:])
    doc_ids = doc_ids.difference(not_query)
    return doc_ids

def boolean_retrieval(query):
    if ' and ' in query:
        query = query.split(' and ')
        doc_ids = boolean_and(query)
    elif ' or ' in query:
        query = query.split(' or ')
        doc_ids = boolean_or(query)
    elif ' not ' in query:
        query = query.split(' not ')
        doc_ids = boolean_not(query)
    else:
        if query in inverted_index:
            doc_ids = set(inverted_index[query])
        else:
            doc_ids = set()
    doc_scores = [(doc_id, cosine_similarities[0][doc_id]) for doc_id in doc_ids]
    doc_scores.sort(key=lambda x: x[1], reverse=True)
    return doc_scores

# Get input query from user
query = input("Enter your query: ")

# Preprocess the query
query = cleaned_text(query)

# Perform boolean retrieval with document IDs and rank
doc_scores = boolean_retrieval(query)

# Print the top 10 documents with doc ID, title, and rank
for i in range(min(10, len(doc_scores))):
    doc_id = doc_scores[i][0]
    title = data.iloc[doc_id]['Title']
    print(f"Rank {i+1}: Document ID {doc_id}, Title '{title}'")

Enter your query: matrix or joker
Rank 1: Document ID 32, Title '33. Joker (2019)'
Rank 2: Document ID 15, Title '16. The Matrix (1999)'


### **Wild Card Query**

In [ ]:
# Clean and preprocess the text
def cleaned_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip() 

# Create the inverted index
inverted_index = {}
for index, row in data.iterrows():
    title = cleaned_text(row['Title'])
    words = title.split()
    doc_id = index
    for word in words:
        if word in inverted_index:
            inverted_index[word].append((doc_id, title))
        else:
            inverted_index[word] = [(doc_id, title)]

def identifyTypeOfWildCardQuery(query,splChar):
    if query.startswith(splChar):
        return (query.split(splChar)[1],1)
    elif query.endswith(splChar):
        return (query.split(splChar)[0],2)
    else:
        temp = query.split(splChar)
        return (temp[0],temp[1])

query = input("Enter a Wild-Card query: ")
query_len = len(query)
resultOfWidlCardSearch = []
if "*" not in query and "?" not in query:
    print("It is not a valid Wild-Card query")
else:
    if "*" in query:
        wildCardQuery = identifyTypeOfWildCardQuery(query,"*")
        for i in inverted_index:
            if wildCardQuery[1] == 2:
                if i.startswith(wildCardQuery[0]):
                    resultOfWidlCardSearch.append(inverted_index[i])
            elif wildCardQuery[1] == 1:
                if i.endswith(wildCardQuery[0]):
                    resultOfWidlCardSearch.append(inverted_index[i])
            elif i.startswith(wildCardQuery[0]) and i.endswith(wildCardQuery[1]):
                resultOfWidlCardSearch.append(inverted_index[i])
    else:
        wildCardQuery = identifyTypeOfWildCardQuery(query,"?")
        for i in inverted_index:
            if wildCardQuery[1] == 2:
                if i.startswith(wildCardQuery[0]) and len(i) == query_len:
                    resultOfWidlCardSearch.append(inverted_index[i])
            elif wildCardQuery[1] == 1:
                if i.endswith(wildCardQuery[0])and len(i) == query_len:
                    resultOfWidlCardSearch.append(inverted_index[i])
            elif i.startswith(wildCardQuery[0]) and i.endswith(wildCardQuery[1]) and len(i) == query_len:
                resultOfWidlCardSearch.append(inverted_index[i])

docIdsOfWildCardQuery = set()
for i in resultOfWidlCardSearch:
    for j in i:
        docIdsOfWildCardQuery.add(j[0])

if docIdsOfWildCardQuery:
    print(f"Document IDs: {docIdsOfWildCardQuery}")
    print("Titles:")
    for doc_id in docIdsOfWildCardQuery:
        print(f"{doc_id}: {data.loc[doc_id, 'Title']}")
else:
    print("No documents found matching the wildcard query.")


Enter a Wild-Card query: godf*
Document IDs: {1, 3}
Titles:
1: 2. The Godfather (1972)
3: 4. The Godfather: Part II (1974)


### **Pharse Query using Bi-word Index and Positional Indexing**



In [ ]:
# Clean and preprocess the text
def cleaned_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip() 

# Create the biword index
biword_index = {}
for index, row in data.iterrows():
    title = cleaned_text(row['Title'])
    words = title.split()
    doc_id = index
    for i in range(len(words)-1):
        biword = words[i] + " " + words[i+1]
        if biword in biword_index:
            biword_index[biword].append((doc_id, i))
        else:
            biword_index[biword] = [(doc_id, i)]

# Create the positional index
positional_index = {}
for index, row in data.iterrows():
    title = cleaned_text(row['Title'])
    words = title.split()
    doc_id = index
    for i, word in enumerate(words):
        if word in positional_index:
            positional_index[word].append((doc_id, i))
        else:
            positional_index[word] = [(doc_id, i)]

# Prompt for user input
query = input("Enter a phrase query: ")
rank = input("Do you want to rank the results by position (y/n)? ")

# Find candidate documents using biword index
query_words = query.lower().split()
candidate_docs = set()
for i in range(len(query_words)-1):
    biword = query_words[i] + " " + query_words[i+1]
    if biword in biword_index:
        for doc_id, pos in biword_index[biword]:
            candidate_docs.add(doc_id)

# Find matching documents and positions using positional index
matched_docs = []
for doc_id in candidate_docs:
    positions = []
    for i, word in enumerate(query_words):
        if i == 0:
            positions = [pos for doc, pos in positional_index[word] if doc == doc_id]
        else:
            new_positions = []
            for pos in positions:
                if (doc_id, pos+i) in positional_index[word]:
                    new_positions.append(pos+i)
            positions = new_positions
        if not positions:
            break
    if positions:
        matched_docs.append((doc_id, positions))

# Sort by number of matching positions if ranking is requested
if rank == 'y':
    if matched_docs:
        matched_docs.sort(key=lambda x: len(x[1]), reverse=True)
else:
    matched_docs.sort()

# Display results
if matched_docs:
    print("Matching documents:")
    for doc_id, positions in matched_docs:
        title = data.iloc[doc_id]['Title']
        print(f"Doc ID: {doc_id}, Title: {title}, Matched positions: {positions}")
else:
    print("No matching documents found.")


Enter a phrase query: the dark
Do you want to rank the results by position (y/n)? y
Matching documents:
Doc ID: 480, Title: 383. Dancer in the Dark (2000), Matched positions: [4]
Doc ID: 2, Title: 3. The Dark Knight (2008), Matched positions: [2]
Doc ID: 580, Title: 383. Dancer in the Dark (2000), Matched positions: [4]
Doc ID: 680, Title: 383. Dancer in the Dark (2000), Matched positions: [4]
Doc ID: 780, Title: 383. Dancer in the Dark (2000), Matched positions: [4]
Doc ID: 880, Title: 383. Dancer in the Dark (2000), Matched positions: [4]
Doc ID: 980, Title: 383. Dancer in the Dark (2000), Matched positions: [4]
Doc ID: 380, Title: 383. Dancer in the Dark (2000), Matched positions: [4]
Doc ID: 62, Title: 63. The Dark Knight Rises (2012), Matched positions: [2]


**Advanced Search: Re-Rank**

In [ ]:
# Preprocess the dataset
data['Title'] = data['Title'].str.lower()
data['Genre'] = data['Genre'].str.lower()
data['Cast'] = data['Cast'].str.lower()

def re_rank_results(query, results):
    """
    Re-rank the search results based on the query.
    
    Args:
    - query (str): the search query
    - results (list of dicts): the search resultsA
    
    Returns:
    - A list of dicts, where each dict represents a search result and contains the keys 'Title', 'Genre', 'Cast', and 'Score'.
      The list is sorted in descending order based on the 'Score' key.
    """
    
    # Define a function to calculate the score of a document based on the query
    def calculate_score(document):
        score = 0
        
        # Calculate the score based on the document title
        if query in document['Title']:
            score += 3
        elif any(word in document['Title'] for word in query.split()):
            score += 1
            
        # Calculate the score based on the document genre
        if query in document['Genre']:
            score += 2
        elif any(word in document['Genre'] for word in query.split()):
            score += 1
            
        # Calculate the score based on the document cast
        if query in document['Cast']:
            score += 2
        elif any(word in document['Cast'] for word in query.split()):
            score += 1
            
        return score
    
    # Calculate the score of each document and add it as a new key in the dictionary
    for document in results:
        document['Score'] = calculate_score(document)
        
    # Sort the results based on the score
    results.sort(key=lambda x: x['Score'], reverse=True)
    
    return results


In [ ]:
query = "action adventure"

In [ ]:
results

[(68, 25.200000000000003),
 (2, 18.0),
 (8, 17.6),
 (20, 17.2),
 (35, 17.0),
 (62, 16.8),
 (118, 16.6),
 (153, 16.4),
 (19, 8.6),
 (27, 8.6),
 (32, 8.5),
 (36, 8.5),
 (40, 8.5),
 (49, 8.5),
 (46, 8.5),
 (66, 8.4),
 (79, 8.4),
 (85, 8.3),
 (86, 8.3),
 (88, 8.3),
 (103, 8.3),
 (110, 8.3),
 (119, 8.3),
 (125, 8.3),
 (144, 8.2),
 (161, 8.2),
 (162, 8.2),
 (168, 8.2),
 (177, 8.2),
 (182, 8.2),
 (183, 8.2),
 (145, 8.2),
 (157, 8.2),
 (192, 8.1),
 (195, 8.1),
 (198, 8.1),
 (205, 8.1),
 (215, 8.1),
 (228, 8.1),
 (237, 8.1),
 (240, 8.1),
 (243, 8.1),
 (250, 8.1),
 (270, 8.1),
 (278, 8.1),
 (284, 8.1),
 (288, 8.1),
 (301, 8.1),
 (302, 8.1),
 (305, 8.1),
 (401, 8.1),
 (402, 8.1),
 (405, 8.1),
 (501, 8.1),
 (502, 8.1),
 (505, 8.1),
 (601, 8.1),
 (602, 8.1),
 (605, 8.1),
 (701, 8.1),
 (702, 8.1),
 (705, 8.1),
 (801, 8.1),
 (802, 8.1),
 (805, 8.1),
 (901, 8.1),
 (902, 8.1),
 (905, 8.1),
 (262, 8.1),
 (220, 8.1),
 (225, 8.1),
 (239, 8.1),
 (273, 8.1),
 (324, 8.0),
 (331, 8.0),
 (333, 8.0),
 (334, 8.0

In [ ]:
# Preprocess the dataset
data['Title'] = data['Title'].str.lower()
data['Genre'] = data['Genre'].str.lower()
data['Cast'] = data['Cast'].str.lower()

def re_rank_results(query, results):
    """
    Re-rank the search results based on the query.

    Args:
    - query (str): the search query
    - results (list of dicts): the search results

    Returns:
    - A list of dicts, where each dict represents a search result and contains the keys 'Title', 'Genre', 'Cast', and 'Score'.
      The list is sorted in descending order based on the 'Score' key.
    """

    # Define a function to calculate the score of a document based on the query
    def calculate_score(document):
        score = 0

        # Calculate the score based on the document title
        if query in document['Title']:
            score += 3
        elif any(word in document['Title'] for word in query.split()):
            score += 1

        # Calculate the score based on the document genre
        if query in document['Genre']:
            score += 2
        elif any(word in document['Genre'] for word in query.split()):
            score += 1

        # Calculate the score based on the document cast
        if query in document['Cast']:
            score += 2
        elif any(word in document['Cast'] for word in query.split()):
            score += 1

        return score

    # Calculate the score of each document and add it as a new key in the dictionary
    for document in results:
        document['Score'] = calculate_score(document)

    # Sort the results based on the score
    results.sort(key=lambda x: x['Score'], reverse=True)

    return results

# Define the input query
query = "the dark knight"

# Perform the search
search_results = []
for index, row in data.iterrows():
    search_results.append({'Title': row['Title'], 'Genre': row['Genre'], 'Cast': row['Cast']})

# Call the re_rank_results function with the query and search results
re_ranked_results = re_rank_results(query, search_results)

# Print the top 5 re-ranked results
for result in re_ranked_results[:5]:
    print(result['Title'], "|", result['Genre'], "|", result['Cast'], "|", result['Score'])


dark knight | action , crime , drama | christian bale , heath ledger , aaron eckhart | 1
godfath | crime , drama | marlon brando , al pacino , jame caan | 0
godfath : part ii | crime , drama | al pacino , robert de niro , robert duval | 0
shawshank redempt | drama | tim robbin , morgan freeman , bob gunton | 0
